In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import spacy

import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from pathlib import Path
from ast import literal_eval
from spacy.tokens import Doc, Span, Token

In [33]:
nlp = spacy.load('en_core_web_sm')

ner = nlp.get_pipe('ner')

for _,annotations in train:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

pipe_exceptions = ['ner', 'trf_wordpiecer', 'trf_tok2vec']
unaffected_pipe = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [32]:

df = pd.read_csv('./data/train_set.csv', sep = ';', encoding = 'utf-8-sig')

lista = df['entidades'].to_list()
lista = [literal_eval(x) for x in lista]

train, test = train_test_split(lista, test_size=0.25, random_state=8)

In [ ]:
nr_iter = 10

with nlp.disable_pipes(*unaffected_pipe):
    for iteration in range(nr_iter):
        random.shuffle(train)
        losses = {}

        batches = minibatch(train, size=8)
        for batch in batches:
            example = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example.append(Example.from_dict(doc, annotations))

            nlp.update(example, drop = 0.3, losses=losses)

        print(f"Iteration {iteration} - Losses: {losses}")


In [ ]:
# Test the model
test_text = "CALLE SIMCA 8, 3B"
doc = nlp(test_text)

print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])